In [2]:
import pathlib
import requests
import subprocess
import os
import shutil
from typing import List, Tuple
from astropy.io import fits
import astropy
import pandas as pd

#### The input_file needs to be formatted properly if you do not use the one created from swift_database_creator.ipynb
Space delimited data with each line containing a target name -> observation id (no commas or any other characters between these elements)   
*Note: This data can have any amount of spacing between the name and observation id, just none* ***before the name*** *and/or* ***after the id***

In [3]:
def create_tlist(input_file: str) -> List[Tuple[str, str]]:
    tlist = []
    file = open(f'{input_file}', "r")
    lines = file.readlines()
    i = 0
    while i < len(lines):
        current_line = lines[i]
        obsid = current_line.split()[-1]
        tname = current_line.split()[0]
        if (tname != 'Name' and tname != '-------------------'):
            tlist.append((obsid, tname))
        i += 1
    file.close()
    return tlist

In [4]:
def create_swift_wget_commands(obsid: str, dtype: str, overwrite: bool=False) -> str:
    
    # -nc ==> no clobber: don't replace already downloaded files
    # -q ==> quiet mode, no output
    # -w 2 ==> wait 2 seconds between files
    # -nH ==> don't create a directory based on the host, in this case no folder named www.swift.ac.uk/
    # --cut-dirs=2 ==> remove the /archive/reproc/ folders on the server from being created locally
    # -r ==> recursive: grab everything under this folder on the server
    # --reject ... ==> specify files that we don't want from the server
    
    if overwrite is False:
        overwrite_option = '-nc'
    else:
        overwrite_option = ''
    wget_command = 'wget ' + overwrite_option + f' -q -w 2 -nH --cut-dirs=2 -r --no-parent --reject index.html*,robots.txt*  http://www.swift.ac.uk/archive/reproc/{obsid}/{dtype}/'
    return wget_command

In [5]:
def swift_download_uncompressed(obsid: str, updated_name: str, dtype: str, dest_dir: pathlib.Path = None, overwrite: bool=False) -> None:
    
    wget_command = create_swift_wget_commands(obsid, dtype, overwrite)
    old_cwd = os.getcwd()
    if dest_dir is not None:
        os.chdir(dest_dir)
    if (overwrite == False and os.path.isdir(f'{os.getcwd()}/{updated_name}_{obsid}/{dtype}') == True):
        print(f'{dtype} data for observation id {obsid} is already found in {os.getcwd()}')
        return
    print(f"Downloading {dtype} data of observation id {obsid} to {os.getcwd()} ...")
    print(wget_command.split(' ')[-1])
    presult = subprocess.run(wget_command.split())
    if presult.returncode != 0:
        print(f"Non-zero return code {presult.returncode} for {wget_command}!")
        return
    if (os.path.isdir(f'{os.getcwd()}/{updated_name}_{obsid}')):
        try:
            shutil.move(f'{os.getcwd()}/{obsid}/{dtype}', f'{os.getcwd()}/{updated_name}_{obsid}/{dtype}')
            shutil.rmtree(f'{os.getcwd()}/{obsid}')
        except FileNotFoundError:
            shutil.rmtree(f'{os.getcwd()}/{obsid}')
    try:
        os.rename(f'{os.getcwd()}/{obsid}', f'{os.getcwd()}/{updated_name}_{obsid}')
        os.chdir(old_cwd)
    except (os.error, FileNotFoundError):
        os.chdir(old_cwd)

In [6]:
def swift_download_compressed() -> None:
    return

In [7]:
def download_data(tlist: str, dtype_list: str, dest_dir: pathlib.Path, download_type: str, overwrite=False) -> None:
    for obsid, tname in tlist:
        updated_name = tname.replace('/', '_')
        if (overwrite == True and os.path.isdir(f'{os.getcwd()}/{updated_name}_{obsid}')):
            shutil.rmtree(f'{os.getcwd()}/{updated_name}_{obsid}')
        for dtype in dtype_list:
            if download_type == 'uncompressed':
                swift_download_uncompressed(obsid=obsid, updated_name=updated_name, dtype=dtype, dest_dir=dest_dir, overwrite=overwrite)
            if download_type == 'tar':
                swift_download_compressed()
    print(f'\nDownload Completed!')

In [8]:
dtype_list = ['auxil', 'bat', 'xrt', 'uvot', 'log']
download_type = ['uncompressed', 'tar']
dest_dir = '/Volumes/Swift Data'
file_name = 'test.txt'
tlist = create_tlist(input_file=file_name)

#download_data(tlist=tlist, dtype_list=dtype_list, dest_dir=dest_dir, download_type=download_type[0], overwrite=False)

In [9]:
def count_uvot_images(tname: str, obsid: str, filter_type: str) -> int:
    updated_name = tname.replace('/', '_')
    try:
        files = os.listdir(f'/Volumes/Swift Data/{updated_name}_{obsid}/uvot/image/')
    except (os.error, FileNotFoundError):
        return 0
    image_paths = []
    for j in files:
        if filter_type in j:
            image_paths.append(f'/Volumes/Swift Data/{updated_name}_{obsid}/uvot/image/{j}')
    if image_paths == None:
        return 0
    count = 0
    for path in image_paths:
        image = fits.open(path)
        for i in image:
            if (isinstance(image[i], astropy.io.fits.hdu.image.ImageHDU)):
                count += 1
    return count

In [10]:
count_uvot_images('Comet73P_3-C','00030682001','_sk')

6

In [17]:
def update_tlist(tlist: list) -> List[Tuple[str, str, int, int, int, int]]:
    new_list = []
    i = 0
    while i < len(tlist):
        sk_count = count_uvot_images(tname=tlist[i][1],obsid=tlist[i][0],filter_type='_sk')
        rw_count = count_uvot_images(tname=tlist[i][1],obsid=tlist[i][0],filter_type='_rw')
        ex_count = count_uvot_images(tname=tlist[i][1],obsid=tlist[i][0],filter_type='_ex')
        total_count = sk_count + rw_count + ex_count
        new_list.append((tlist[i][0], tlist[i][1], sk_count, rw_count, ex_count, total_count))
        i += 1
        if (i % 100 == 0):
            print(f'{i/len(tlist)} percent remaining.')
    return new_list

In [18]:
file_name = 'swift_comet_database.txt'
tlist = create_tlist(input_file=file_name)
new_list = update_tlist(tlist)
#print(new_list)

0.031928480204342274 percent remaining.
0.06385696040868455 percent remaining.
0.09578544061302682 percent remaining.
0.1277139208173691 percent remaining.
0.15964240102171137 percent remaining.
0.19157088122605365 percent remaining.
0.22349936143039592 percent remaining.
0.2554278416347382 percent remaining.
0.28735632183908044 percent remaining.
0.31928480204342274 percent remaining.
0.351213282247765 percent remaining.
0.3831417624521073 percent remaining.
0.41507024265644954 percent remaining.
0.44699872286079184 percent remaining.
0.4789272030651341 percent remaining.
0.5108556832694764 percent remaining.
0.5427841634738186 percent remaining.
0.5747126436781609 percent remaining.
0.6066411238825032 percent remaining.
0.6385696040868455 percent remaining.
0.6704980842911877 percent remaining.
0.70242656449553 percent remaining.
0.7343550446998723 percent remaining.
0.7662835249042146 percent remaining.
0.7982120051085568 percent remaining.
0.8301404853128991 percent remaining.
0.86

In [21]:
os.chdir('/Users/jduffy0121/Desktop/')
fname = '/Users/jduffy0121/Downloads/swift_comet_database_normalized.csv'
comet_dataframe = pd.read_csv(filepath_or_buffer=fname)
sorted_list = sorted(new_list, key=lambda tobs: int(tobs[0]))
comet_dataframe.sort_values('obsid', inplace=True)
comet_dataframe.reset_index(drop=True, inplace=True)
print(len(comet_dataframe))
print(len(sorted_list))
#print(comet_dataframe)
#print(sorted_list[-5:])
#print(comet_dataframe[:5])
#print(sorted_list[0])
comet_dataframe.insert(
    loc=3,
    column="sk_image_count",
    value=[x[2] for x in sorted_list],
)
comet_dataframe.insert(
    loc=4,
    column="rw_image_count",
    value=[x[3] for x in sorted_list],
)
comet_dataframe.insert(
    loc=5,
    column="ex_image_count",
    value=[x[4] for x in sorted_list],
)
comet_dataframe.insert(
    loc=6,
    column="total_image_count",
    value=[x[5] for x in sorted_list],
)
comet_dataframe.to_csv('/Users/jduffy0121/Downloads/swift_comet_database_test_main.csv', index=False)

3132
3132


In [ ]:
#or j in sorted_list:
#    if int(j[0]) == int('30396001'):
#        print(j[2])
print(sorted_list)

In [24]:
sw_mask = comet_dataframe['conventional_names'] == '73P'
print(sum(comet_dataframe[sw_mask]['total_image_count']))

2654
